In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import sqlalchemy
import json
config=json.load(open("/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/config.json","r"))
username=config['username']
password=config['password']
database=config['database']

BL_engine=sqlalchemy.create_engine(
            "mysql+pymysql://%s:%s@localhost/%s" % (username, password, database))
print(datetime.datetime.now())
os.getcwd()

2020-09-02 11:26:39.626870


'/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL'

In [2]:
folder_output='/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/'
list_folder_output_by_week=os.listdir(folder_output)
list_folder_output_by_week=[folder_output+x+"/" for x in list_folder_output_by_week if "output_No_DCM_" in x]
list_folder_output_by_week.sort()
list_folder_output_by_week

['/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-05-02_2020-08-24/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-05-09_2020-08-23/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-05-16_2020-08-23/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-05-23_2020-08-22/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-05-30_2020-08-21/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-06-06_2020-08-20/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-06-13_2020-08-17/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-06-20_2020-08-17/',
 '/home/jian/Projects/Big_Lots/Predictive_Model/extract_from_MySQL/output_No_DCM_2020-06-27_2020-08-16/',
 '/home/jian/Projects/Big_Lots/Predictive_Mode

In [3]:
list_folder_output_by_week=[x for x in list_folder_output_by_week if "/output_No_DCM_2020-07-25_" not in x]

In [1]:
folder_unsubsription='/home/sharefolder/biglots_data/unsubscribe/'
list_unsubsription_files=glob.glob(folder_unsubsription+"*.csv")
list_unsubsription_files=[x for x in list_unsubsription_files if "iber_File_Refresh__" in x]
list_unsubsription_files.sort()
list_unsubsription_files

NameError: name 'glob' is not defined

In [24]:
df_output_train_summary=pd.DataFrame()
df_output_test_summary=pd.DataFrame()
df_output_both_summary=pd.DataFrame()

for weekly_folder in list_folder_output_by_week:
    files=os.listdir(weekly_folder)
    
    file_trans_1_only_DV3=[weekly_folder+x for x in files if "BL_LRModeling_NoDCM_trans_1_only_DV3" in x][0]
    file_trans_2_plus_DV2=[weekly_folder+x for x in files if "BL_LRModeling_NoDCM_trans_2_plus_DV2" in x][0]
    week_end_dt=file_trans_1_only_DV3.split("_JL_")[0][-10:]
    week_end_dt=datetime.datetime.strptime(week_end_dt,"%Y-%m-%d").date()
    
    df_unsub_files=pd.DataFrame({"file_path":list_unsubsription_files})
    df_unsub_files['date']=df_unsub_files['file_path'].apply(lambda x: x.split("ile_Refresh__")[1][:8])
    df_unsub_files['date']=pd.to_datetime(df_unsub_files['date']).dt.date
    df_unsub_files['day_diff']=abs(df_unsub_files['date']-week_end_dt)
    path_unsub=df_unsub_files[df_unsub_files['day_diff']==df_unsub_files['day_diff'].min()]['file_path'].values.tolist()[0]
    list_unsunsribe_ids=pd.read_csv(path_unsub,
                             dtype=str,usecols=['customersummary_c_primaryscnhash'])['customersummary_c_primaryscnhash'].unique().tolist()
    
    sql_str_start="'"+str(week_end_dt-datetime.timedelta(days=13))+"'"
    sql_str_end="'"+str(week_end_dt)+"'"
    df_ids_shoppers_past_2_weeks=pd.read_sql("select distinct customer_id_hashed as customer_id_hashed from Pred_POS_Department where transaction_dt between %s and %s and customer_id_hashed is not null;"%(sql_str_start,sql_str_end),con=BL_engine)
    df_ids_shoppers_past_2_weeks['past_shoppers']="shopper_in_past_2_weeks"
    df_week_train=pd.DataFrame()
    df_week_test=pd.DataFrame()
    df_week_both=pd.DataFrame()
    
    # train
    for file in [file_trans_1_only_DV3,file_trans_2_plus_DV2]:
        print(week_end_dt,file.split("BL_LRModeling_NoDCM_")[1][:27],datetime.datetime.now())
        type_model=file.split("BL_LRModeling_NoDCM_")[1][:12]
        DV_n=file.split(type_model)[1][1:4]
        excel_file=pd.ExcelFile(file)
        file_ids_train=[weekly_folder+x for x in files if "df_train" in x and type_model in x][0]
        file_ids_test=[weekly_folder+x for x in files if "df_test" in x and type_model in x][0]
        ### train
        df_ids_train=pd.read_csv(file_ids_train)
        df_ids_train['subscription_lables']=np.where(df_ids_train['customer_id_hashed'].isin(list_unsunsribe_ids),"unsub","default_subs")

        df_ids_train=pd.merge(df_ids_train,df_ids_shoppers_past_2_weeks,on="customer_id_hashed",how="left")
        count_target_ids_shoppered_past2weeks_train=df_ids_train[(df_ids_train['selection_label']=="target") & (df_ids_train['past_shoppers']=="shopper_in_past_2_weeks")]
        count_target_ids_shoppered_past2weeks_train=len(count_target_ids_shoppered_past2weeks_train)
        count_nonselect_ids_shoppered_past2weeks_train=df_ids_train[(df_ids_train['selection_label']=="nonselect") & (df_ids_train['past_shoppers']=="shopper_in_past_2_weeks")]
        count_nonselect_ids_shoppered_past2weeks_train=len(count_nonselect_ids_shoppered_past2weeks_train)
        ### test
        df_ids_test=pd.read_csv(file_ids_test)
        df_ids_test['subscription_lables']=np.where(df_ids_test['customer_id_hashed'].isin(list_unsunsribe_ids),"unsub","default_subs")

        df_ids_test=pd.merge(df_ids_test,df_ids_shoppers_past_2_weeks,on="customer_id_hashed",how="left")
        count_target_ids_shoppered_past2weeks_test=df_ids_test[(df_ids_test['selection_label']=="target") & (df_ids_test['past_shoppers']=="shopper_in_past_2_weeks")]
        count_target_ids_shoppered_past2weeks_test=len(count_target_ids_shoppered_past2weeks_test)
        count_nonselect_ids_shoppered_past2weeks_test=df_ids_test[(df_ids_test['selection_label']=="nonselect") & (df_ids_test['past_shoppers']=="shopper_in_past_2_weeks")]
        count_nonselect_ids_shoppered_past2weeks_test=len(count_nonselect_ids_shoppered_past2weeks_test)
        
        # new to file past 4 weeks count
        count_id_train_new_signs_past4weeks=df_ids_train[(df_ids_train['sign_up_label']=="new_signs")].shape[0]
        count_id_train_new_signs_past4weeks_sub=df_ids_train[(df_ids_train['sign_up_label']=="new_signs") & (df_ids_train['subscription_lables']=="default_subs")].shape[0]
        count_id_train_new_signs_past4weeks_unsub=df_ids_train[(df_ids_train['sign_up_label']=="new_signs") & (df_ids_train['subscription_lables']=="unsub")].shape[0]
        
        count_id_test_new_signs_past4weeks=df_ids_test[(df_ids_test['sign_up_label']=="new_signs")].shape[0]
        count_id_test_new_signs_past4weeks_sub=df_ids_test[(df_ids_test['sign_up_label']=="new_signs") & (df_ids_test['subscription_lables']=="default_subs")].shape[0]
        count_id_test_new_signs_past4weeks_unsub=df_ids_test[(df_ids_test['sign_up_label']=="new_signs") & (df_ids_test['subscription_lables']=="unsub")].shape[0]
        
        
        # count
        df=excel_file.parse("df_dataset_shape")
        train_records=df.loc[df['Unnamed: 0']=="X_train","records"].values[0]
        test_records=df.loc[df['Unnamed: 0']=="X_test","records"].values[0]

        # cutoff
        df=excel_file.parse("select_score_matrix")
        cutoff=df.iloc[0,0]
        # Media selection
        # Train
        df_train=excel_file.parse("train_id_summary")
        target_count_train=df_train[df_train['selection_label']=="target"]['customer_id_hashed'].sum()
        nonselect_count_train=df_train[df_train['selection_label']=="nonselect"]['customer_id_hashed'].sum()
        
        target_count_train_new=df_train[(df_train['selection_label']=="target") & (df_train['sign_up_label']=="new_signs")]['customer_id_hashed'].sum()
        target_count_train_existing=df_train[(df_train['selection_label']=="target") & (df_train['sign_up_label']=="existing")]['customer_id_hashed'].sum()
        target_count_train_existing_sub=df_ids_train[(df_ids_train['selection_label']=="target") & (df_ids_train['sign_up_label']=="existing") & (df_ids_train['subscription_lables']=="default_subs")].shape[0]
        target_count_train_existing_unsub=df_ids_train[(df_ids_train['selection_label']=="target") & (df_ids_train['sign_up_label']=="existing") & (df_ids_train['subscription_lables']=="unsub")].shape[0]
        nonselect_count_train_existing_sub=df_ids_train[(df_ids_train['selection_label']=="nonselect") & (df_ids_train['sign_up_label']=="existing") & (df_ids_train['subscription_lables']=="default_subs")].shape[0]
        nonselect_count_train_existing_unsub=df_ids_train[(df_ids_train['selection_label']=="nonselect") & (df_ids_train['sign_up_label']=="existing") & (df_ids_train['subscription_lables']=="unsub")].shape[0]        
        
        shoper_count_train_target=df_train[(df_train['selection_label']=="target") & (df_train['actual_shopping_label']=="shopper")]['customer_id_hashed'].sum()
        shoper_count_train_nonselect=df_train[(df_train['selection_label']=="nonselect") & (df_train['actual_shopping_label']=="shopper")]['customer_id_hashed'].sum()

        target_shop_rate_train=np.round(shoper_count_train_target/target_count_train,4)
        nonselect_shop_rate_train=np.round(shoper_count_train_nonselect/nonselect_count_train,4)
        target_shop_rate_P2W_train=np.round(count_target_ids_shoppered_past2weeks_train/target_count_train,4)
        nonselect_shop_rate_P2W_train=np.round(count_nonselect_ids_shoppered_past2weeks_train/nonselect_count_train,4)
        
        # Test
        df_test=excel_file.parse("test_id_summary")
        target_count_test=df_test[df_test['selection_label']=="target"]['customer_id_hashed'].sum()
        nonselect_count_test=df_test[df_test['selection_label']=="nonselect"]['customer_id_hashed'].sum()
        total_count_test=target_count_test+nonselect_count_test

        target_count_test_new=df_test[(df_test['selection_label']=="target") & (df_test['sign_up_label']=="new_signs")]['customer_id_hashed'].sum()
        target_count_test_existing=df_test[(df_test['selection_label']=="target") & (df_test['sign_up_label']=="existing")]['customer_id_hashed'].sum()
        target_count_test_existing_sub=df_ids_test[(df_ids_test['selection_label']=="target") & (df_ids_test['sign_up_label']=="existing") &(df_ids_test['subscription_lables']=="default_subs")].shape[0]
        target_count_test_existing_unsub=df_ids_test[(df_ids_test['selection_label']=="target") & (df_ids_test['sign_up_label']=="existing") & (df_ids_test['subscription_lables']=="unsub")].shape[0]
        nonselect_count_test_existing_sub=df_ids_test[(df_ids_test['selection_label']=="nonselect") & (df_ids_test['sign_up_label']=="existing") & (df_ids_test['subscription_lables']=="default_subs")].shape[0]
        nonselect_count_test_existing_unsub=df_ids_test[(df_ids_test['selection_label']=="nonselect") & (df_ids_test['sign_up_label']=="existing") & (df_ids_test['subscription_lables']=="unsub")].shape[0]        
        
        
        shoper_count_test_target=df_test[(df_test['selection_label']=="target") & (df_test['actual_shopping_label']=="shopper")]['customer_id_hashed'].sum()
        shoper_count_test_nonselect=df_test[(df_test['selection_label']=="nonselect") & (df_test['actual_shopping_label']=="shopper")]['customer_id_hashed'].sum()
        
        target_shop_rate_test=np.round(shoper_count_test_target/target_count_test,4)
        nonselect_shop_rate_test=np.round(shoper_count_test_nonselect/nonselect_count_test,4)
        target_shop_rate_P2W_test=np.round(count_target_ids_shoppered_past2weeks_test/target_count_test,4)
        nonselect_shop_rate_P2W_test=np.round(count_nonselect_ids_shoppered_past2weeks_test/nonselect_count_test,4)
        
        # Both
        target_shop_rate_DV_both=np.round(sum([shoper_count_train_target,shoper_count_test_target])/sum([target_count_train,target_count_test]),4)
        nonselect_shop_rate_DV_both=np.round(sum([shoper_count_train_nonselect,shoper_count_test_nonselect])/sum([nonselect_count_train,nonselect_count_test]),4)
        target_shop_rate_P2W_both=np.round(sum([count_target_ids_shoppered_past2weeks_train,count_target_ids_shoppered_past2weeks_test])/sum([target_count_train,target_count_test]),4)
        nonselect_shop_rate_P2W_both=np.round(sum([count_nonselect_ids_shoppered_past2weeks_train,count_nonselect_ids_shoppered_past2weeks_test])/sum([target_count_train,nonselect_count_test]),4)
        
        df_output_train=pd.DataFrame({
            "Total_Counts":train_records,
            "cutoffs":cutoff,
            "target_count":target_count_train,
            "target_new_ids":target_count_train_new,
            "past4W_new_ids_subset":count_id_train_new_signs_past4weeks,
            "past4W_new_ids_subset_sub":count_id_train_new_signs_past4weeks_sub,
            "past4W_new_ids_subset_unsub":count_id_train_new_signs_past4weeks_unsub,
            "target_existing_ids":target_count_train_existing,
            "target_existing_ids_sub":target_count_train_existing_sub,
            "target_existing_ids_unsub":target_count_train_existing_unsub,
            "target_shop_rate_DV":target_shop_rate_train,
            "nonselect_shop_rate_DV":nonselect_shop_rate_train,
            "target_shop_rate_Past2W":target_shop_rate_P2W_train,
            "nonselect_shop_rate_Past2W":nonselect_shop_rate_P2W_train,
            'nonselect_count_existing_sub':nonselect_count_train_existing_sub,
            'nonselect_count_existing_unsub':nonselect_count_train_existing_unsub
        },index=[type_model]).T.rename(columns={type_model:week_end_dt}).reset_index()
        
        df_output_test=pd.DataFrame({
            "Total_Counts":test_records,
            "cutoffs":cutoff,
            "target_count":target_count_test,
            "target_new_ids":target_count_test_new,
            "past4W_new_ids_subset":count_id_test_new_signs_past4weeks,
            "past4W_new_ids_subset_sub":count_id_test_new_signs_past4weeks_sub,
            "past4W_new_ids_subset_unsub":count_id_test_new_signs_past4weeks_unsub,
            "target_existing_ids":target_count_test_existing,
            "target_existing_ids_sub":target_count_test_existing_sub,
            "target_existing_ids_unsub":target_count_test_existing_unsub,
            "target_shop_rate_DV":target_shop_rate_test,
            "nonselect_shop_rate_DV":nonselect_shop_rate_test,
            "target_shop_rate_Past2W":target_shop_rate_P2W_test,
            "nonselect_shop_rate_Past2W":nonselect_shop_rate_P2W_test,
            'nonselect_count_existing_sub':nonselect_count_test_existing_sub,
            'nonselect_count_existing_unsub':nonselect_count_test_existing_unsub
        },index=[type_model]).T.rename(columns={type_model:week_end_dt}).reset_index()

        

    
        # iterrate the cutoff of 1 trans
        if type_model=="trans_1_only":
            df_train_id_new=df_ids_train[df_ids_train['sign_up_label']=="new_signs"]
            df_new_cutoff_train=pd.DataFrame()        
            for i in range(1,6):
                new_cutoff=np.round(cutoff/(1+i/10),4)
                df_train_id_new['selection_label']=np.where(df_train_id_new['y_hat']>=new_cutoff,"target","nonselect")
                df=pd.DataFrame({week_end_dt:df_train_id_new['selection_label'].tolist().count("target")},index=["count_new_target_at_%d0pctg_diminishing"%i])
                df_new_cutoff_train=df_new_cutoff_train.append(df)
                
            df_test_id_new=df_ids_test[df_ids_test['sign_up_label']=="new_signs"]
            df_new_cutoff_test=pd.DataFrame()        
            for i in range(1,6):
                new_cutoff=np.round(cutoff/(1+i/10),4)
                df_test_id_new['selection_label']=np.where(df_test_id_new['y_hat']>=new_cutoff,"target","nonselect")
                df=pd.DataFrame({week_end_dt:df_test_id_new['selection_label'].tolist().count("target")},index=["count_new_target_at_%d0pctg_diminishing"%i])
                df_new_cutoff_test=df_new_cutoff_test.append(df)  
            df_new_cutoff_train=df_new_cutoff_train.reset_index()
            df_new_cutoff_test=df_new_cutoff_test.reset_index()
            df_output_train=df_output_train.append(df_new_cutoff_train)
            df_output_test=df_output_test.append(df_new_cutoff_test)
            
                
        elif type_model=="trans_2_plus":
            df_train_id_all=df_ids_train.copy()
            df_update_cutoff_train=pd.DataFrame()        
            for i in range(1,6):
                updated_cutoff=np.round(cutoff/(1+i/10),4)
                df_train_id_all['selection_label']=np.where(df_train_id_all['y_hat']>=updated_cutoff,"target","nonselect")
                df=pd.DataFrame({week_end_dt:df_train_id_all['selection_label'].tolist().count("target")},index=["count_updated_target_at_%d0pctg_diminishing"%i])
                df_update_cutoff_train=df_update_cutoff_train.append(df)
                
            df_test_id_all=df_ids_test.copy()
            df_update_cutoff_test=pd.DataFrame()        
            for i in range(1,6):
                updated_cutoff=np.round(cutoff/(1+i/10),4)
                df_test_id_all['selection_label']=np.where(df_test_id_all['y_hat']>=updated_cutoff,"target","nonselect")
                df=pd.DataFrame({week_end_dt:df_test_id_all['selection_label'].tolist().count("target")},index=["count_updated_target_at_%d0pctg_diminishing"%i])
                df_update_cutoff_test=df_update_cutoff_test.append(df)  
            
            df_update_cutoff_train=df_update_cutoff_train.reset_index()
            df_update_cutoff_test=df_update_cutoff_test.reset_index()
            
            df_output_train=df_output_train.append(df_update_cutoff_train)
            df_output_test=df_output_test.append(df_update_cutoff_test)
            
            
        df_output_train['type_model']=type_model
        df_output_train['DV_type']=DV_n
        df_output_test['type_model']=type_model
        df_output_test['DV_type']=DV_n
        
        df_output_train=df_output_train.set_index(['index','DV_type','type_model'])
        df_output_test=df_output_test.set_index(['index','DV_type','type_model'])
        
        df_week_train=df_week_train.append(df_output_train)
        df_week_test=df_week_test.append(df_output_test)
        

        df_output_total=pd.DataFrame({
            "Total_Counts":train_records+test_records,
            "cutoffs":cutoff,
            "target_count":target_count_train+target_count_test,
            "target_new_ids":target_count_train_new+target_count_test_new,
            "past4W_new_ids_subset":count_id_train_new_signs_past4weeks+count_id_test_new_signs_past4weeks,
            "past4W_new_ids_subset_sub":count_id_train_new_signs_past4weeks_sub+count_id_test_new_signs_past4weeks_sub,
            "past4W_new_ids_subset_unsub":count_id_train_new_signs_past4weeks_unsub+count_id_test_new_signs_past4weeks_unsub,
            "target_existing_ids":target_count_train_existing+target_count_test_existing,
            "target_existing_ids_sub":target_count_train_existing_sub+target_count_test_existing_sub,
            "target_existing_ids_unsub":target_count_train_existing_unsub+target_count_test_existing_unsub,
            "target_shop_rate_DV":target_shop_rate_DV_both,
            "nonselect_shop_rate_DV":nonselect_shop_rate_DV_both,
            "target_shop_rate_Past2W":target_shop_rate_P2W_both,
            "nonselect_shop_rate_Past2W":nonselect_shop_rate_P2W_both,
            'nonselect_count_existing_sub':nonselect_count_train_existing_sub+nonselect_count_test_existing_sub,
            'nonselect_count_existing_unsub':nonselect_count_train_existing_unsub+nonselect_count_test_existing_unsub
        },index=[type_model]).T.rename(columns={type_model:week_end_dt}).reset_index()
        df_output_total['type_model']=type_model
        df_output_total['DV_type']=DV_n
        df_output_total=df_output_total.set_index(['index','DV_type','type_model'])
        df_week_both=df_week_both.append(df_output_total)
        
    
    if len(df_output_train_summary)==0:
        df_output_train_summary=df_week_train
    else:
        df_output_train_summary=pd.concat([df_output_train_summary,df_week_train],axis=1)
        
    if len(df_output_test_summary)==0:
        df_output_test_summary=df_week_test
    else:
        df_output_test_summary=pd.concat([df_output_test_summary,df_week_test],axis=1)
        
    if len(df_output_both_summary)==0:
        df_output_both_summary=df_week_both
    else:
        df_output_both_summary=pd.concat([df_output_both_summary,df_week_both],axis=1)


2020-05-02 trans_1_only_DV3_2020-05-02 2020-09-02 14:46:45.274861


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2020-05-02 trans_2_plus_DV2_2020-05-02 2020-09-02 14:47:04.616662
2020-05-09 trans_1_only_DV3_2020-05-09 2020-09-02 14:51:26.529120
2020-05-09 trans_2_plus_DV2_2020-05-09 2020-09-02 14:51:45.379915
2020-05-16 trans_1_only_DV3_2020-05-16 2020-09-02 14:56:09.992297
2020-05-16 trans_2_plus_DV2_2020-05-16 2020-09-02 14:56:28.605826
2020-05-23 trans_1_only_DV3_2020-05-23 2020-09-02 15:00:58.437651
2020-05-23 trans_2_plus_DV2_2020-05-23 2020-09-02 15:01:17.629468
2020-05-30 trans_1_only_DV3_2020-05-30 2020-09-02 15:05:49.109233
2020-05-30 trans_2_plus_DV2_2020-05-30 2020-09-02 15:06:08.525740
2020-06-06 trans_1_only_DV3_2020-06-06 2020-09-02 15:10:11.367364
2020-06-06 trans_2_plus_DV2_2020-06-06 2020-09-02 15:10:30.339112
2020-06-13 trans_1_only_DV3_2020-06-13 2020-09-02 15:15:31.739010
2020-06-13 trans_2_plus_DV2_2020-06-13 2020-09-02 15:15:51.330823
2020-06-20 trans_1_only_DV3_2020-06-20 2020-09-02 15:19:50.595787
2020-06-20 trans_2_plus_DV2_2020-06-20 2020-09-02 15:20:08.924349
2020-06-27

In [25]:
df_output_train_summary=df_output_train_summary.reset_index()
df_output_test_summary=df_output_test_summary.reset_index()
df_output_both_summary=df_output_both_summary.reset_index()

df_train_new_update=df_output_train_summary[~df_output_train_summary['index'].isin(df_output_both_summary['index'].tolist())]
df_test_new_update=df_output_test_summary[~df_output_test_summary['index'].isin(df_output_both_summary['index'].tolist())]
df_train_new_update=df_train_new_update.set_index(['index','DV_type','type_model'])
df_test_new_update=df_test_new_update.set_index(['index','DV_type','type_model'])

df_both_new_update=df_train_new_update+df_test_new_update
df_both_new_update=df_both_new_update.reset_index()
df_output_both_summary=df_output_both_summary.append(df_both_new_update)
df_order=df_output_train_summary[['index','DV_type','type_model']]
df_order['order']=range(df_order.shape[0])

df_output_both_summary=pd.merge(df_order,df_output_both_summary,on=['index','DV_type','type_model'],how="outer")
df_output_both_summary=df_output_both_summary.sort_values("order")
del df_output_both_summary['order']

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
writer=pd.ExcelWriter("./BL_RL_modeling_summary_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output_both_summary.to_excel(writer,"overall",index=True)
df_output_train_summary.to_excel(writer,"train",index=True)
df_output_test_summary.to_excel(writer,"test",index=True)
writer.save()